# OpenAI tests for lambda - azure function converter

## Testing Chain of Thought Prompting (CoT)

### Install the necessary libraries

In [ ]:
pip install langchain

In [ ]:
pip install openai

In [ ]:
pip install chromadb

In [ ]:
pip install tiktoken

In [ ]:
pip install python-dotenv

### Setup

Import the libraries and environment variables to gain access to the `Open API Key`

In [1]:
import os
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain.vectorstores import Chroma
from langchain.embeddings import AzureOpenAIEmbeddings
from langchain.prompts import FewShotChatMessagePromptTemplate, ChatPromptTemplate
from langchain.chat_models import AzureChatOpenAI

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

api_key=os.environ['OPENAI_API_KEY']
base_url=os.environ['OPENAI_BASE_URL']

print(base_url)

https://devsquad-eastus-2.openai.azure.com/


Let's create the llm client, we can use a chat llm from langchain

In [2]:
llm = AzureChatOpenAI(
    api_key=api_key,
    azure_endpoint=base_url, 
    api_version="2023-07-01-preview",
    model="gpt-4",
    temperature=0
)

#### Create embeddings of your documents to get ready for semantic search

We are going to get our embeddings engine ready. This will be the engine that will turn out documents into vector embeddings so we can easily do semantic search on them.

In [3]:
embeddings = AzureOpenAIEmbeddings(
    api_key=api_key,
    azure_endpoint=base_url, 
    api_version="2023-07-01-preview",
    azure_deployment="text-embedding-ada-002"
)

In [4]:
text = "this is a test document"
query_result = embeddings.embed_query(text)
doc_result = embeddings.embed_documents([text])
doc_result[0][:5]

[-0.012222584727053142,
 0.007210398239221619,
 -0.014818063280923785,
 -0.026444746872933574,
 -0.003433049970082691]

### Create the example set
To get started, create a list of few-shot examples. Each example should be a dictionary with the keys being the input variables and the values being the values for those input variables.

**TODO:** Add metadata to examples

In [5]:
examples = [
    {
        "input": '''
package main

import (
	"context"
	"fmt"
	"github.com/aws/aws-lambda-go/lambda"
)

type MyEvent struct {
	Name string `json:"name"`
}

type MyResponse struct {
	Message string `json:"message"`
}

func HandleRequest(ctx context.Context, event *MyEvent) (*string, error) {
	if event == nil {
		return nil, fmt.Errorf("received nil event")
	}
	message := fmt.Sprintf("Hello %s!", event.Name)
	return &MyResponse{Message: message}, nil
}

func main() {
	lambda.Start(HandleRequest)
}
        ''',
        "output": '''
package main

import (
 "net/http"
 "os"

 "github.com/gin-gonic/gin"
)

const (
	EnvVarAzureFunctionPort string = "FUNCTIONS_PORT"
)

type MyEvent struct {
	Name string `json:"name"`
}

type MyResponse struct {
	Message string `json:"message"`
}

func HandleRequest(ctx *gin.Context) {
    var event MyEvent
	err = c.Bind(&event)
	if err != nil {
		fmt.Printf("error on reading request body: %v\n", err.Error())
		ctx.JSON(http.StatusBadRequest, gin.H{"error": err.Error()})
		return
	}
	if event == nil {
        errorMsg := "received nil event"    
		fmt.Printf(errorMsg)
		ctx.JSON(http.StatusBadRequest, gin.H{"error": errorMsg})
		return
	}
	message := fmt.Sprintf("Hello %s!", event.Name)
    ctx.JSON(http.StatusOK, &MyResponse{Message: message})
}


func main() {
 r := gin.Default()
 r.Handle(http.MethodPost, "/HandleRequest", HandleRequest)

 port := os.LookupEnv(EnvVarAzureFunctionPort)
 host := fmt.Sprintf("0.0.0.0:%s", port)
 fmt.Println("Go server Listening...on port: ", port)
 log.Panic(r.Run(host))
}
        '''
    }
]

### Create a formatter for the few-shot examples
Configure a formatter that will format the few-shot examples into a string. This formatter should be a `PromptTemplate` object.

In [6]:
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)

In [7]:
print(example_prompt.format(**examples[0]))

Human: 
package main

import (
	"context"
	"fmt"
	"github.com/aws/aws-lambda-go/lambda"
)

type MyEvent struct {
	Name string `json:"name"`
}

type MyResponse struct {
	Message string `json:"message"`
}

func HandleRequest(ctx context.Context, event *MyEvent) (*string, error) {
	if event == nil {
		return nil, fmt.Errorf("received nil event")
	}
	message := fmt.Sprintf("Hello %s!", event.Name)
	return &MyResponse{Message: message}, nil
}

func main() {
	lambda.Start(HandleRequest)
}
        
AI: 
package main

import (
 "net/http"
 "os"

 "github.com/gin-gonic/gin"
)

const (
	EnvVarAzureFunctionPort string = "FUNCTIONS_PORT"
)

type MyEvent struct {
	Name string `json:"name"`
}

type MyResponse struct {
	Message string `json:"message"`
}

func HandleRequest(ctx *gin.Context) {
    var event MyEvent
	err = c.Bind(&event)
	if err != nil {
		fmt.Printf("error on reading request body: %v
", err.Error())
		ctx.JSON(http.StatusBadRequest, gin.H{"error": err.Error()})
		return
	}
	if event =

### Feed examples and formatter to FewShotPromptTemplate
Finally, create a `FewShotPromptTemplate` object. This object takes in the few-shot examples and the formatter for the few-shot examples.

In [8]:
input = '''
package main

import (
	"context"
	"fmt"
	"github.com/aws/aws-lambda-go/lambda"
	"github.com/devsquad/saving"
)

type SaveRequest struct {
	Id string `json:"id"`
}

type Response struct {
	Message string `json:"message"`
}

func HandleRequest(ctx context.Context, req *SaveRequest) (*string, error) {
	if req == nil {
		return nil, fmt.Errorf("request can't be nil")
	}
	err := saving.Save(ctx, req.id)
	message := fmt.Sprintf("request %%s was successfully saved", req.id)
	return &Response{{Message: message}}, nil
}

func main() {
	lambda.Start(HandleRequest)
}               
'''

print(input)


package main

import (
	"context"
	"fmt"
	"github.com/aws/aws-lambda-go/lambda"
	"github.com/devsquad/saving"
)

type SaveRequest struct {
	Id string `json:"id"`
}

type Response struct {
	Message string `json:"message"`
}

func HandleRequest(ctx context.Context, req *SaveRequest) (*string, error) {
	if req == nil {
		return nil, fmt.Errorf("request can't be nil")
	}
	err := saving.Save(ctx, req.id)
	message := fmt.Sprintf("request %%s was successfully saved", req.id)
	return &Response{{Message: message}}, nil
}

func main() {
	lambda.Start(HandleRequest)
}               



In [9]:
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

print(few_shot_prompt.format())

Human: 
package main

import (
	"context"
	"fmt"
	"github.com/aws/aws-lambda-go/lambda"
)

type MyEvent struct {
	Name string `json:"name"`
}

type MyResponse struct {
	Message string `json:"message"`
}

func HandleRequest(ctx context.Context, event *MyEvent) (*string, error) {
	if event == nil {
		return nil, fmt.Errorf("received nil event")
	}
	message := fmt.Sprintf("Hello %s!", event.Name)
	return &MyResponse{Message: message}, nil
}

func main() {
	lambda.Start(HandleRequest)
}
        
AI: 
package main

import (
 "net/http"
 "os"

 "github.com/gin-gonic/gin"
)

const (
	EnvVarAzureFunctionPort string = "FUNCTIONS_PORT"
)

type MyEvent struct {
	Name string `json:"name"`
}

type MyResponse struct {
	Message string `json:"message"`
}

func HandleRequest(ctx *gin.Context) {
    var event MyEvent
	err = c.Bind(&event)
	if err != nil {
		fmt.Printf("error on reading request body: %v
", err.Error())
		ctx.JSON(http.StatusBadRequest, gin.H{"error": err.Error()})
		return
	}
	if event =

### Using Advanced Chain-of-Thought (CoT) Prompt

Add a system input that explains the steps the LLM needs to follow

In [10]:
system_input = '''You are an azure assistant that converts aws lambda code into azure functions. 
    You do it by using the following steps:
    
    Step 1. Determine if the code corresponds to a lambda function, by checking its imports
    Step 2. If it isn't a lambda function, return it as is
    Step 3. If it is a lambda function, convert to an azure function
   
   Return the resultant code.'''

In [11]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_input),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

print(final_prompt.format(input=input))

System: You are an azure assistant that converts aws lambda code into azure functions. 
                          You do it by using the following steps:
                          
   Step 1. Determine if the code corresponds to a lambda function, by checking its imports
   Step 2. If it isn't a lambda function, return it as is
   Step 3. If it is a lambda function, convert to an azure function
   
   Return the resultant code.
Human: 
package main

import (
	"context"
	"fmt"
	"github.com/aws/aws-lambda-go/lambda"
)

type MyEvent struct {
	Name string `json:"name"`
}

type MyResponse struct {
	Message string `json:"message"`
}

func HandleRequest(ctx context.Context, event *MyEvent) (*string, error) {
	if event == nil {
		return nil, fmt.Errorf("received nil event")
	}
	message := fmt.Sprintf("Hello %s!", event.Name)
	return &MyResponse{Message: message}, nil
}

func main() {
	lambda.Start(HandleRequest)
}
        
AI: 
package main

import (
 "net/http"
 "os"

 "github.com/gin-gonic/g

### Using an example selector

We will reuse the example set and the formatter from the previous section. However, instead of feeding the examples directly into the `FewShotPromptTemplate` object, we will feed them into an `ExampleSelector` object.

In this tutorial, we will use the `SemanticSimilarityExampleSelector` class. This class selects few-shot examples based on their similarity to the input. It uses an embedding model to compute the similarity between the input and the few-shot examples, as well as a vector store to perform the nearest neighbor search.

In [12]:
to_vectorize = [" ".join(example.values()) for example in examples]
vectorstore = Chroma.from_texts(to_vectorize, embeddings, metadatas=examples)

In [13]:
example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vectorstore,
    k=2,
)

# The prompt template will load examples by passing the input do the `select_examples` method
example_selector.select_examples({"input": input})

Number of requested results 2 is greater than number of elements in index 1, updating n_results = 1


[{'input': '\npackage main\n\nimport (\n\t"context"\n\t"fmt"\n\t"github.com/aws/aws-lambda-go/lambda"\n)\n\ntype MyEvent struct {\n\tName string `json:"name"`\n}\n\ntype MyResponse struct {\n\tMessage string `json:"message"`\n}\n\nfunc HandleRequest(ctx context.Context, event *MyEvent) (*string, error) {\n\tif event == nil {\n\t\treturn nil, fmt.Errorf("received nil event")\n\t}\n\tmessage := fmt.Sprintf("Hello %s!", event.Name)\n\treturn &MyResponse{Message: message}, nil\n}\n\nfunc main() {\n\tlambda.Start(HandleRequest)\n}\n        ',
  'output': '\npackage main\n\nimport (\n "net/http"\n "os"\n\n "github.com/gin-gonic/gin"\n)\n\nconst (\n\tEnvVarAzureFunctionPort string = "FUNCTIONS_PORT"\n)\n\ntype MyEvent struct {\n\tName string `json:"name"`\n}\n\ntype MyResponse struct {\n\tMessage string `json:"message"`\n}\n\nfunc HandleRequest(ctx *gin.Context) {\n    var event MyEvent\n\terr = c.Bind(&event)\n\tif err != nil {\n\t\tfmt.Printf("error on reading request body: %v\n", err.Error

### Feed example selector into FewShotPromptTemplate
Finally, create a `FewShotPromptTemplate` object. This object takes in the example selector and the formatter for the few-shot examples.

In [14]:
from langchain.prompts import (
    ChatPromptTemplate,
    FewShotChatMessagePromptTemplate,
)

# Define the few-shot prompt.
few_shot_prompt = FewShotChatMessagePromptTemplate(
    # The input variables select the values to pass to the example_selector
    input_variables=["input"],
    example_selector=example_selector,
    # Define how each example will be formatted.
    # In this case, each example will become 2 messages:
    # 1 human, and 1 AI
    example_prompt=ChatPromptTemplate.from_messages(
        [("human", "{input}"), ("ai", "{output}")]
    ),
)

print(few_shot_prompt.format(input=input))

Number of requested results 2 is greater than number of elements in index 1, updating n_results = 1


Human: 
package main

import (
	"context"
	"fmt"
	"github.com/aws/aws-lambda-go/lambda"
)

type MyEvent struct {
	Name string `json:"name"`
}

type MyResponse struct {
	Message string `json:"message"`
}

func HandleRequest(ctx context.Context, event *MyEvent) (*string, error) {
	if event == nil {
		return nil, fmt.Errorf("received nil event")
	}
	message := fmt.Sprintf("Hello %s!", event.Name)
	return &MyResponse{Message: message}, nil
}

func main() {
	lambda.Start(HandleRequest)
}
        
AI: 
package main

import (
 "net/http"
 "os"

 "github.com/gin-gonic/gin"
)

const (
	EnvVarAzureFunctionPort string = "FUNCTIONS_PORT"
)

type MyEvent struct {
	Name string `json:"name"`
}

type MyResponse struct {
	Message string `json:"message"`
}

func HandleRequest(ctx *gin.Context) {
    var event MyEvent
	err = c.Bind(&event)
	if err != nil {
		fmt.Printf("error on reading request body: %v
", err.Error())
		ctx.JSON(http.StatusBadRequest, gin.H{"error": err.Error()})
		return
	}
	if event =

In [15]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_input),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

print(final_prompt.format(input=input))

Number of requested results 2 is greater than number of elements in index 1, updating n_results = 1


System: You are an azure assistant that converts aws lambda code into azure functions. 
                          You do it by using the following steps:
                          
   Step 1. Determine if the code corresponds to a lambda function, by checking its imports
   Step 2. If it isn't a lambda function, return it as is
   Step 3. If it is a lambda function, convert to an azure function
   
   Return the resultant code.
Human: 
package main

import (
	"context"
	"fmt"
	"github.com/aws/aws-lambda-go/lambda"
)

type MyEvent struct {
	Name string `json:"name"`
}

type MyResponse struct {
	Message string `json:"message"`
}

func HandleRequest(ctx context.Context, event *MyEvent) (*string, error) {
	if event == nil {
		return nil, fmt.Errorf("received nil event")
	}
	message := fmt.Sprintf("Hello %s!", event.Name)
	return &MyResponse{Message: message}, nil
}

func main() {
	lambda.Start(HandleRequest)
}
        
AI: 
package main

import (
 "net/http"
 "os"

 "github.com/gin-gonic/g

### Use with an LLM
Now, you can connect your model to the few-shot prompt.

In [16]:
result = llm(final_prompt.format_messages(input=input))

Number of requested results 2 is greater than number of elements in index 1, updating n_results = 1


In [17]:
print(result.content)

The provided code snippet is a Go AWS Lambda function that imports the `aws-lambda-go/lambda` package, which indicates that it is indeed a Lambda function. The function `HandleRequest` is designed to handle a `SaveRequest` and uses a custom package `saving` to perform a save operation. 

To convert this AWS Lambda function into an Azure Function, we need to adapt the code to the Azure Functions runtime for Go. Azure Functions in Go can be created using custom handlers, which allow you to define HTTP-triggered functions using any Go web framework. Below is the converted code using the Gin web framework, which is a popular choice for creating web applications and APIs in Go.

```go
package main

import (
	"fmt"
	"net/http"
	"os"

	"github.com/devsquad/saving"
	"github.com/gin-gonic/gin"
)

type SaveRequest struct {
	Id string `json:"id"`
}

type Response struct {
	Message string `json:"message"`
}

func HandleRequest(c *gin.Context) {
	var req SaveRequest
	if err := c.BindJSON(&req); err

### Let's try with format instructions

It's possible to tell our LLM how we expect to get our response back using output parsers, let's try it out

### Check if compiles 

Here the idea would be to compile the classes and if it doesn't compile tell the LLM the issue and retry

### Testing more inputs

Try again using new inputs

In [18]:
lambda_code_1 = '''
package main

import (
	"context"
	"encoding/json"
	"github.com/aws/aws-lambda-go/events"
	runtime "github.com/aws/aws-lambda-go/lambda"
	"github.com/aws/aws-lambda-go/lambdacontext"
	"github.com/aws/aws-sdk-go/aws/session"
	"github.com/aws/aws-sdk-go/service/lambda"
	"log"
	"os"
)

var client = lambda.New(session.New())

func callLambda() (string, error) {
	input := &lambda.GetAccountSettingsInput{}
	req, resp := client.GetAccountSettingsRequest(input)
	err := req.Send()
	output, _ := json.Marshal(resp.AccountUsage)
	return string(output), err
}

func handleRequest(ctx context.Context, event events.SQSEvent) (string, error) {
	// event
	eventJson, _ := json.MarshalIndent(event, "", "  ")
	log.Printf("EVENT: %s", eventJson)
	// environment variables
	log.Printf("REGION: %s", os.Getenv("AWS_REGION"))
	log.Println("ALL ENV VARS:")
	for _, element := range os.Environ() {
		log.Println(element)
	}
	// request context
	lc, _ := lambdacontext.FromContext(ctx)
	log.Printf("REQUEST ID: %s", lc.AwsRequestID)
	// global variable
	log.Printf("FUNCTION NAME: %s", lambdacontext.FunctionName)
	// context method
	deadline, _ := ctx.Deadline()
	log.Printf("DEADLINE: %s", deadline)
	// AWS SDK call
	usage, err := callLambda()
	if err != nil {
		return "ERROR", err
	}
	return usage, nil
}

func main() {
	runtime.Start(handleRequest)
}
'''

In [19]:
result = llm(final_prompt.format_messages(input=lambda_code_1))

Number of requested results 2 is greater than number of elements in index 1, updating n_results = 1


In [20]:
print(result.content)

```go
package main

import (
	"context"
	"encoding/json"
	"log"
	"net/http"
	"os"

	"github.com/Azure/azure-sdk-for-go/sdk/messaging/azservicebus"
	"github.com/Azure/azure-sdk-for-go/sdk/messaging/azservicebus/admin"
	"github.com/Azure/azure-sdk-for-go/sdk/resourcemanager/resources/armresources"
	"github.com/Azure/azure-sdk-for-go/sdk/azidentity"
	"github.com/gin-gonic/gin"
)

func callServiceBus() (string, error) {
	cred, err := azidentity.NewDefaultAzureCredential(nil)
	if err != nil {
		log.Fatalf("Failed to obtain a credential: %v", err)
	}
	client, err := admin.NewClient(os.Getenv("SERVICEBUS_NAMESPACE"), cred, nil)
	if err != nil {
		log.Fatalf("Failed to create client: %v", err)
	}

	queueProps, err := client.GetQueueRuntimeProperties(context.Background(), os.Getenv("SERVICEBUS_QUEUE"), nil)
	if err != nil {
		log.Fatalf("Failed to get queue properties: %v", err)
	}

	output, _ := json.Marshal(queueProps)
	return string(output), err
}

func handleRequest(ctx *gin.Context) {
	var